## Imports

In [11]:
import pandas as pd

## Dataset -> Dataframes

In [15]:
IL_colleges_path = "../datasets/illinois/IL_nyt_colleges.csv"
IPEDS_allIL_path = "../datasets/illinois/ipeds/ipeds.csv"

df_nyt_IL_colleges = pd.read_csv(IL_colleges_path)
df_ipeds_allIL_colleges = pd.read_csv(IPEDS_allIL_path)